In [0]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time

In [0]:
# Load for desktop
path_to_file = "wonderland.txt"


In [0]:
# load for internet or link
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [0]:

text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print ('Length of text: {} characters'.format(len(text)))
print(text[:250])
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

Length of text: 9246 characters
৬১টি সংস্থার তহবিলের উদ্বৃত্ত টাকা সরকারি কোষাগারে জমা নেওয়ার জন্য বিল পাস করাতে গিয়ে জাতীয় সংসদে বিরোধী দলের তীব্র বিরোধিতা ও সমালোচনার মুখে পড়েছেন অর্থমন্ত্রী আ হ ম মুস্তফা কামাল।

বিএনপি ও জাতীয় পার্টির একাধিক সাংসদ এই আইনকে ‘কালো আইন’ আখ্যা দিয়
79 unique characters


In [0]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [0]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:19]))

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}
'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53]


In [0]:
seq_length = 100
examples_per_epoch = len(text)//seq_length
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(15):
  print(idx2char[i.numpy()])

F
i
r
s
t
 
C
i
t
i
z
e
n
:




In [0]:

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [0]:

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [0]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [0]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:6], target_example[:6])):
#for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    #print("Step {:4d}".format(i))
    print("Step {:5d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step     0
  input: 18 ('F')
  expected output: 47 ('i')
Step     1
  input: 47 ('i')
  expected output: 56 ('r')
Step     2
  input: 56 ('r')
  expected output: 57 ('s')
Step     3
  input: 57 ('s')
  expected output: 58 ('t')
Step     4
  input: 58 ('t')
  expected output: 1 (' ')
Step     5
  input: 1 (' ')
  expected output: 15 ('C')


In [0]:
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [0]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [0]:
if tf.test.is_gpu_available():
  rnn = tf.keras.layers.CuDNNGRU
else:
  import functools
  rnn = functools.partial(
    tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[batch_size, None]),
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:

model = build_model(
  vocab_size = len(vocab), 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

In [0]:
for input_example_batch, target_example_batch in dataset.take(1): 
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3935232   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,018,497
Trainable params: 4,018,497
Non-trainable params: 0
_________________________________________________________________


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[1], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [0]:
sampled_indices

array([50, 10, 26, 25, 35, 26, 59, 53, 28, 47,  5, 48, 20,  6, 50, 35, 55,
       45, 33,  1,  6,  8, 41, 21, 61, 24, 45,  7, 11, 44, 22, 54,  3, 21,
       27, 62, 48,  3, 34, 20, 44,  9, 36, 64,  8, 40, 52, 23, 47,  0, 31,
       18, 30, 16, 17,  5, 24, 47, 42, 34, 29, 33, 51, 17, 57, 19, 32, 51,
       52, 11, 23, 50, 60, 19, 36,  2, 28, 56, 61, 60, 59, 32, 39, 28, 48,
        9, 11,  8, 25, 29, 58, 33, 33,  3, 13, 27,  5, 33,  6, 28])

In [0]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 "the lord protector's mind herein?\nWho is most inward with the royal duke?\n\nBISHOP OF ELY:\nYour grace"

Next Char Predictions: 
 "l:NMWNuoPi'jH,lWqgU ,.cIwLg-;fJp$IOxj$VHf3Xz.bnKi\nSFRDE'LidVQUmEsGTmn;KlvGX!PrwvuTaPj3;.MQtUU$AO'U,P"


In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.1747766


In [0]:
model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [0]:
EPOCHS=3
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/3
174/174 [==============================] - 1533s 9s/step - loss: 2.6738
Epoch 2/3
174/174 [==============================] - 1525s 9s/step - loss: 1.9346
Epoch 3/3
174/174 [==============================] - 1529s 9s/step - loss: 1.6804


In [0]:
tf.train.latest_checkpoint(checkpoint_dir)
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3935232   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 4,018,497
Trainable params: 4,018,497
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string):
  num_generate = 1000
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []
  temperature = 0.5
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
      input_eval = tf.expand_dims([predicted_id], 0)
      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: he'e the purpose thou art not be to be the tran,
And will be more than the provongerous and hath a proples,
To part them make of the princes of the wind,
That can shall be the store and you have madam then his strange.

GLOUCESTER:
The brother's than the best of the surponce of the rays,
And have the manishand the stronger with that say the dound,
And that thou than the senither's rumbs drath the hands;
And make the stanger was the world warth in hands.

GREMIO:
Stand your lard, and the entery of herpaint.

SABESTIS:
And, loves you be apperied thee, bedome have dead the seased from his best her leave so mand and the him to her.

KING RICHARD III:
Which strick to be than the hath of give with my brood,
And we have be the maids and deed three trought,
That is this suce and the present of my parther,
Her dound the glounded of these to destreming
That say the crounds and wordes here, and brother?

CAMELLO:
So have them no brother will brow him
What was good many and spear to the sho